In [100]:
import spacy
import random
nlp = spacy.load('en_core_web_sm')


In [106]:
def get_entities(sentence, ent_dict):
    for i, token in enumerate(sentence):
        if token.pos_ != 'PART' and token.ent_type_:
            if token.ent_type_ not in ent_dict:
                ent_dict[token.ent_type_] = []
            if token in ent_dict:
                continue
            ent_dict[token.ent_type_].append(token)

In [160]:
def binaryQuestion(sentence, ent_dict):
    aux = None
    tokens = [t for t in sentence]
    #print("=====================")
    #print(doc)
    
    for i, token in enumerate(sentence):
        #print(token.text, token.ent_type_, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
        #print(token.text, token.ent_type_)
        #print(token, token.head, [t for t in token.subtree])
        if token.pos_ == 'AUX':
            aux = token
            if aux in tokens:
                tokens.remove(aux)
        if token.pos_ == 'SCONJ':
            sconj_tree = token.head.subtree
            tokens = [t for t in tokens if t not in token.head.subtree]
            #print("deleting sconj: ", [t for t in token.head.subtree])
        if token.dep_ == 'appos':
            #print("deleting apposition: ", [t for t in token.subtree])
            tokens = [t for t in tokens if t not in token.subtree]
    
    #print(ent_dict)
    
    if not aux:
        #print("No AUX")
        return
    if len(tokens) > 15:
        return
    
    # tweak question by substituting named entities
    def tweak_question():
        for i, t in enumerate(tokens):
            if not (t.pos_ != 'PART' and t.ent_type_):
                continue
            ind = (random.random() * (len(ent_dict[t.ent_type_]) - 1))
            ind = int(ind)
            if not t.pos_ == ent_dict[t.ent_type_][ind].pos_:
                continue
            tokens[i], ent_dict[t.ent_type_][ind] = ent_dict[t.ent_type_][ind], tokens[i]
    
    # generate random seed to decide if we want to tweak question
    if random.random() < 0.66:
        tweak_question()
        print("=======tweaked question!=======")
    else:
        print("=======untweaked question=======")
    
    # Re-capitalize the sentence
    texts = []
    aux_text = aux.text[0].upper() + aux.text[1:]
    for t in tokens:
        if t.pos_ == 'PUNCT':
            continue
        if t.is_sent_start and t.pos_ != 'PROPN':
            texts.append(t.text.lower())
        else:
            texts.append(t.text)
    return [aux_text] + texts + ['?']

In [146]:
text = "Basketball is a team sport in which two teams, most commonly of five players each, opposing one another on a rectangular court, compete with the primary objective of shooting a basketball (approximately 9.4 inches (24 cm) in diameter) through the defender's hoop (a basket 18 inches (46 cm) in diameter mounted 10 feet (3.048 m) high to a backboard at each end of the court) while preventing the opposing team from shooting through their own hoop. A field goal is worth two points, unless made from behind the three-point line, when it is worth three. After a foul, timed play stops and the player fouled or designated to shoot a technical foul is given one, two or three one-point free throws. The team with the most points at the end of the game wins, but if regulation play expires with the score tied, an additional period of play (overtime) is mandated."

In [137]:
text = "The film, which is the third instalment in the Harry Potter film series, was written by Steve Kloves and produced by Chris Columbus (director of the first two instalments), David Heyman, and Mark Radcliffe. The story follows Harry Potter's third year at Hogwarts as he is informed that a prisoner named Sirius Black has escaped from Azkaban intending to kill him."

In [161]:
sentences = [s for s in nlp(text).sents]
ent_dict = {}
for sentence in sentences:
    get_entities(sentence, ent_dict)
print(ent_dict)


{'CARDINAL': [two, five, 3.048, two, three, three, two, three], 'QUANTITY': [approximately, 9.4, inches, 24, cm, 18, inches, 46, cm, 10, feet]}


In [162]:
for sentence in sentences:
    ans = binaryQuestion(sentence, ent_dict)
    if ans:
        print(" ".join([t.text for t in sentence]))
        print("==>" + " ".join(ans))
        print()

=======tweaked question!=======
A field goal is worth two points , unless made from behind the three - point line , when it is worth three .
==>Is a field goal worth three points ?

